In [3]:
import os
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkContext
from pyspark.sql import Row

In [5]:
sc = SparkContext()

In [2]:
full_ratings_file = os.path.join('/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest', 'ratings.csv')

In [6]:
full_ratings_raw_data = sc.textFile(full_ratings_file)

In [7]:
full_ratings_raw_data_header = full_ratings_raw_data.take(1)[0]

In [37]:
full_ratings_data = full_ratings_raw_data.filter(lambda line: line!=full_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()

In [27]:
full_ratings_data.take(3)

[(1, 110, 1.0), (1, 147, 4.5), (1, 858, 5.0)]

In [10]:
full_movies_file = os.path.join('/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest', 'movies.csv')

full_movies_raw_data = sc.textFile(full_movies_file)
full_movies_raw_data_header = full_movies_raw_data.take(1)[0]

full_movies_data = full_movies_raw_data.filter(lambda line: line!=full_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
full_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [40]:
training_RDD, validation_RDD, test_RDD = full_ratings_data.randomSplit([0.6, 0.2, 0.2], 12345)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [44]:
from pyspark.mllib.recommendation import ALS
import math

seed = 2345
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ( 'For rank %s the RMSE is %s' % (rank, error) )
    if error < min_error:
        min_error = error
        best_rank = rank

print ( 'The best model was trained with rank %s' % best_rank )

For rank 4 the RMSE is 0.8348012977310346
For rank 8 the RMSE is 0.8228989471273379
For rank 12 the RMSE is 0.8209800093084014
The best model was trained with rank 12


In [46]:
print ("There are %s recommendations in the complete dataset" % (full_ratings_data.count()))

There are 26024289 recommendations in the complete dataset


In [48]:
training_RDD, test_RDD = full_ratings_data.randomSplit([7, 3], seed=3455)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)

In [50]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.8196448408351055


In [53]:
complete_movies_file = os.path.join('/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print ("There are %s movies in the complete dataset" % (complete_movies_titles.count()))

There are 45843 movies in the complete dataset


In [56]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (full_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [59]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,4), # Star Wars (1977)
     (0,1,3), # Toy Story (1995)
     (0,16,3), # Casino (1995)
     (0,25,4), # Leaving Las Vegas (1995)
     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,1), # Flintstones, The (1994)
     (0,379,1), # Timecop (1994)
     (0,296,3), # Pulp Fiction (1994)
     (0,858,5) , # Godfather, The (1972)
     (0,50,4) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print ('New user ratings: %s' % new_user_ratings_RDD.take(10))

New user ratings: [(0, 260, 4), (0, 1, 3), (0, 16, 3), (0, 25, 4), (0, 32, 4), (0, 335, 1), (0, 379, 1), (0, 296, 3), (0, 858, 5), (0, 50, 4)]


In [60]:
complete_data_with_new_ratings_RDD = full_ratings_data.union(new_user_ratings_RDD)

In [63]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ("New model trained in %s seconds" % round(tt,3))

New model trained in 236.807 seconds


In [64]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [65]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(144300, ((2.3770043675118693, 'As the Light Goes Out (2014)'), 9)),
 (7215, ((3.159584255664488, 'To Have and Have Not (1944)'), 1026)),
 (165945, ((2.1404603219469434, 'The Last Ronin (2010)'), 2))]

In [66]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

In [67]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
('"Godfather', 3.9186008445674614, 57070)
("Long Night's Journey Into Day (2000)", 3.8786773086265605, 35)
('Frozen Planet (2011)', 3.846752095730538, 322)
('"Godfather: Part II', 3.826471688126546, 36679)
('"I', 3.8033975084557583, 63)
('Pulp Fiction (1994)', 3.8026332416173103, 87901)
('"Two Escobars', 3.79200120588933, 63)
('"Cremator', 3.75771395973311, 47)
('Death on the Staircase (Soupçons) (2004)', 3.7410217724557113, 95)
('Planet Earth (2006)', 3.7168898481545423, 754)
('Heimat - A Chronicle of Germany (Heimat - Eine deutsche Chronik) (1984)', 3.713282087151729, 32)
("One Flew Over the Cuckoo's Nest (1975)", 3.713080389556067, 40103)
("Schindler's List (1993)", 3.7129118573053814, 67662)
('HyperNormalisation (2016)', 3.7095668269354047, 61)
('Wanderers (2014)', 3.705188631062464, 59)
('Milius (2013)', 3.7024652890486682, 37)
('"Not on Your Life (Verdugo', 3.698341085342292, 26)
('"Shawshank Redemption', 3.69457493168954, 91082

In [70]:
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3', 'movie_lens_als')

# Save and load model
model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)